In [1]:
import os
import re
import csv
import random
from glob import glob

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torchvision.transforms as v2
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.cli import tqdm

import warnings
warnings.filterwarnings('ignore')


In [2]:
def seed_everything(seed=42):
    random.seed(seed)  # Python 내장 random 모듈
    os.environ['PYTHONHASHSEED'] = str(seed)  # 환경변수 설정
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU 시드 고정
    torch.cuda.manual_seed(seed)  # PyTorch GPU 시드 고정
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU 환경에서도 시드 고정
    torch.backends.cudnn.deterministic = True  # CuDNN 관련 설정
    torch.backends.cudnn.benchmark = False  # 동일한 입력 크기의 데이터가 반복될 경우 속도 향상을 위한 벤치마크 모드 비활성화

# 사용 예시
seed_everything(seed=42)

이미지 경로 포함

In [4]:
valid_df = pd.read_csv('../../../origin_datasets/val_label.csv')
valid_df['data_path'] = valid_df.apply(lambda row: f"../new_image_dataset/user{row['subject_id']}_{row['date']}_valid_fixedY-axis.png", axis=1)
valid_df['subject_id'] = valid_df['subject_id'].apply(lambda x: f'user{str(x).zfill(2)}_valid')

In [8]:
valid_df.head()

,subject_id,date,Q1,Q2,Q3,S1,S2,S3,S4,data_path
0,user01_valid,2023-08-20,1,1,1,0,0,0,0,../new_image_dataset/user1_2023-08-20_valid_fi...
1,user01_valid,2023-08-21,1,1,1,0,0,1,0,../new_image_dataset/user1_2023-08-21_valid_fi...
2,user01_valid,2023-08-22,0,1,1,0,1,1,0,../new_image_dataset/user1_2023-08-22_valid_fi...
3,user01_valid,2023-08-23,0,1,1,0,0,1,0,../new_image_dataset/user1_2023-08-23_valid_fi...
4,user01_valid,2023-08-24,1,1,1,0,0,1,0,../new_image_dataset/user1_2023-08-24_valid_fi...


In [9]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(valid_df, test_size = 0.2, random_state=1020)
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [10]:
print(f"count >>> train_set: {len(train_df)}, valid_set: {len(valid_df)}")

count >>> train_set: 84, valid_set: 21


In [11]:
class CustomDataset(Dataset):
    def __init__(self, df, transforms):
        self.path = df['data_path'].values
        self.class_ = df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3', 'S4']].values
        self.transform = transforms

    def __getitem__(self, idx):
        try:
            img = np.array(Image.open(self.path[idx]).convert('RGB'))
        except FileNotFoundError:
            return self.__getitem__((idx + 1) % len(self))
        
        img = self.transform(image=img)['image']
        img = torch.tensor(img, dtype=torch.float)  # Explicitly specify the type as float
        y = self.class_[idx]
        
        return img, y
    
    def __len__(self):
        return len(self.path)

### 데이터 mean, std 계산

In [12]:
# import os
# import numpy as np
# import cv2
# import albumentations as A

# # 이미지가 저장된 폴더 경로
# folder_path = '../new_image_dataset'

# # 이미지 파일 목록 가져오기
# image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# # 모든 이미지를 읽어 numpy 배열에 저장
# images = []
# for file in image_files:
#     image_path = os.path.join(folder_path, file)
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 흑백 이미지로 읽기
#     if image is not None:
#         images.append(image)

# # 이미지 배열을 numpy 배열로 변환
# images_array = np.array(images)

# # 각 픽셀의 평균과 표준편차 계산
# mean = np.mean(images_array)
# std = np.std(images_array)

# # 결과 출력
# print(f"Mean: {mean}, Std: {std}")

mean = 15.359733188267395
std = 58.28444875927197


In [13]:
import numpy as np
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.pytorch import ToTensorV2

class RandomHorizontalStrips(ImageOnlyTransform):
    def __init__(self, num_strips=(1, 3), strip_width=84, always_apply=False, p=0.5):
        super(RandomHorizontalStrips, self).__init__(always_apply, p)
        self.num_strips = num_strips
        self.strip_width = strip_width

    def apply(self, img, **params):
        h, w = img.shape[:2]
        num_strips = np.random.randint(self.num_strips[0], self.num_strips[1] + 1)
        for _ in range(num_strips):
            x_start = np.random.randint(0, w - self.strip_width)
            img[:, x_start:x_start + self.strip_width] = 0
        return img

    def get_transform_init_args_names(self):
        return ("num_strips", "strip_width")

    def get_params(self):
        return {"num_strips": self.num_strips, "strip_width": self.strip_width}

# 기존의 train_transforms에 새로운 증강 기법 추가
train_transforms = A.Compose([
    # RandomHorizontalStrips(p=0.5),
    A.Resize(height=224, width=224, p=1),
    A.GaussNoise(p=0.5),
    A.OneOf([
        A.GaussianBlur(p=0.5),
        A.Sharpen(p=0.5)
    ], p=0.5),
    A.Normalize(mean=[mean, mean, mean], std=[std, std, std], p=1.0),
    ToTensorV2()
])


test_transforms = A.Compose([
    A.Resize(always_apply = True, p=1.0, height=224, width=224),
    A.Normalize(mean=[mean, mean, mean], std=[std, std, std], p=1.0),
    ToTensorV2()
])

In [14]:
train_dataset = CustomDataset(train_df, train_transforms)
valid_dataset = CustomDataset(valid_df, test_transforms)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [17]:
epochs = 500

In [19]:
from collections import deque

def run_model(model, loader, loss_fn=None, optimizer=None, is_training=False, epoch=None):
    targets = []
    preds = []
    smooth_loss_queue = deque(maxlen=50)  # 최근 50개의 손실을 저장할 큐

    if is_training:
        model.train()
        mode = 'Train'
    else:
        model.eval()
        mode = 'Valid/Test'

    running_loss = 0.0
    bar = tqdm(loader, ascii=True, leave=False)
    for cnt, (data, target) in enumerate(bar):
        data = data.to('cuda')
        target = target.to('cuda')
        if is_training:
            optimizer.zero_grad()

        outputs = torch.sigmoid(model(data))
        total_loss = loss_fn(outputs, target.float())
        running_loss += total_loss.item()
        smooth_loss_queue.append(total_loss.item())  # 현재 손실을 큐에 추가
        smooth_loss = sum(smooth_loss_queue) / len(smooth_loss_queue)  # 큐에 있는 손실의 평균 계산

        predicted = (outputs > 0.5).float()
        preds.extend(predicted.detach().cpu().tolist())
        targets.extend(target.detach().cpu().tolist())

        if is_training:
            total_loss.backward()
            optimizer.step()

        # 에폭 정보와 함께 배치별 손실 평균 및 smooth loss 출력
        bar.set_description(f'Epoch {epoch} {mode} - Loss: {total_loss:.4f}, Smooth Loss: {smooth_loss:.4f}')

    f1_score_ = f1_score(np.array(targets), np.array(preds), average='macro')
    acc_score = accuracy_score(np.array(targets).reshape(-1), np.array(preds).reshape(-1))

    return running_loss / len(loader), acc_score, f1_score_


In [21]:
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [22]:
full_df = pd.read_csv('../../../origin_datasets/val_label.csv')
full_df['Q3'] = full_df['Q3'].apply(lambda x: 1 if x == 0 else 0)
full_df['data_path'] = full_df.apply(lambda row: f"../new_image_dataset/user{row['subject_id']}_{row['date']}_valid_all_sensor.png", axis=1)

In [ ]:
import logging
# 데이터셋 클래스와 변환, 모델, 손실 함수, 최적화 기법을 그대로 사용
# 위 코드에서 정의한 CustomDataset, train_transforms, test_transforms 등을 사용

# 전체 데이터셋 준비
full_dataset = CustomDataset(full_df, train_transforms)  # full_df는 전체 데이터프레임을 나타냅니다.

# KFold 설정
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=1020)

# K-fold 교차 검증 시작
fold_perf = {}

for fold, (train_idx, valid_idx) in enumerate(kf.split(full_dataset)):
    print(f"Fold {fold+1}")
    
    # 데이터셋 분할
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)
    
    # 데이터 로더 설정
    train_loader = DataLoader(full_dataset, batch_size=16, sampler=train_subsampler, num_workers=12)
    valid_loader = DataLoader(full_dataset, batch_size=16, sampler=valid_subsampler, num_workers=12)
    
    # 모델 초기화 및 이동
    model = timm.create_model('resnext101_32x32d', pretrained=True, num_classes=7)
    model = model.to('cuda')
    
    # 손실 함수, 최적화, 스케줄러 설정
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=0)
    scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=100, T_mult=2, eta_max=5e-5,  T_up=10, gamma=0.5)
    
    best_f1 = -float('inf')  # 이 fold에서 최고 F1-score 초기화
    best_loss = 0
    # 학습 및 검증
    for epoch in range(epochs):
        train_loss, train_acc, train_f1 = run_model(model, train_loader, criterion, optimizer, is_training=True, epoch=epoch)
        valid_loss, valid_acc, valid_f1 = run_model(model, valid_loader, criterion, optimizer, is_training=False, epoch=epoch)
        print(f'Epoch: {epoch+1} | Train Loss: {train_loss:.6f} | Train Acc: {train_acc:.6f} | Train F1: {train_f1:.6f} | Valid Loss: {valid_loss:.6f} | Valid Acc: {valid_acc:.6f} | Valid F1: {valid_f1:.6f} | LR: {optimizer.param_groups[0]["lr"]:.2e}')
        # F1-score를 기준으로 모델 저장
        if valid_f1 > best_f1:
            logging.info(f'Epoch {epoch}: New best F1-score {valid_f1:.4f} in fold {fold+1}')
            best_f1 = valid_f1
            best_loss = valid_loss
            print(f'Epoch {epoch}: New best F1-score {valid_f1:.4f} in fold {fold+1}, model saved.')
        
        scheduler.step()
    model_path = f'model_fold_{fold}_f1-{best_f1:.3f}_loss-{best_loss:.3f}_reverseQ3.pt'
    torch.save(model.state_dict(), model_path)
    logging.info(f'Model saved: {model_path}')
    # 각 fold의 성능 기록
    fold_perf[fold] = {
        'train_loss': train_loss,
        'train_acc': train_acc,
        'train_f1': train_f1,
        'valid_loss': valid_loss,
        'valid_acc': valid_acc,
        'valid_f1': valid_f1
    }


Fold 1


Epoch: 1 | Train Loss: 0.718149 | Train Acc: 0.479592 | Train F1: 0.476848 | Valid Loss: 0.715973 | Valid Acc: 0.544218 | Valid F1: 0.436211 | LR: 0.00e+00
Epoch 0: New best F1-score 0.4362 in fold 1, model saved.


Epoch: 2 | Train Loss: 0.680869 | Train Acc: 0.568027 | Train F1: 0.563089 | Valid Loss: 0.783091 | Valid Acc: 0.408163 | Valid F1: 0.227226 | LR: 5.00e-06


Epoch: 3 | Train Loss: 0.592503 | Train Acc: 0.688776 | Train F1: 0.665031 | Valid Loss: 1.407294 | Valid Acc: 0.374150 | Valid F1: 0.261035 | LR: 1.00e-05


Epoch: 4 | Train Loss: 0.503086 | Train Acc: 0.775510 | Train F1: 0.737921 | Valid Loss: 1.577389 | Valid Acc: 0.374150 | Valid F1: 0.261035 | LR: 1.50e-05


...

Epoch: 275 | Train Loss: 0.001361 | Train Acc: 1.000000 | Train F1: 1.000000 | Valid Loss: 0.903780 | Valid Acc: 0.721088 | Valid F1: 0.665161 | LR: 4.84e-07


Epoch: 276 | Train Loss: 0.001884 | Train Acc: 1.000000 | Train F1: 1.000000 | Valid Loss: 0.858014 | Valid Acc: 0.700680 | Valid F1: 0.645021 | LR: 4.26e-07


Epoch: 277 | Train Loss: 0.004157 | Train Acc: 1.000000 | Train F1: 1.000000 | Valid Loss: 0.989222 | Valid Acc: 0.693878 | Valid F1: 0.627342 | LR: 3.71e-07


Epoch: 278 | Train Loss: 0.002416 | Train Acc: 1.000000 | Train F1: 1.000000 | Valid Loss: 0.767835 | Valid Acc: 0.707483 | Valid F1: 0.646589 | LR: 3.20e-07


Epoch 278 Train - Loss: 0.0010, Smooth Loss: 0.0010:  17% 1/6 [00:01<00:05,  1.12s/it]

In [ ]:
import timm

resnet_models = timm.list_models()

for model_name in resnet_models:
    print(model_name)